In [1]:
import gpxpy
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import haversine_distances
import geopandas as gpd
from shapely.geometry import LineString
import osmnx as ox
import folium

In [84]:
def read_simple_gpx(path):
    with open(path, 'r', encoding='utf-8') as f:
        gpx = gpxpy.parse(f)
    rows = []
    for track in gpx.tracks:
        for segment in track.segments:
            for ind, point in enumerate(segment.points):
                row = dict(
                    name=getattr(track, "name", None),
                    step=ind,
                    lat=point.latitude,
                    lon=point.longitude,
                    elevation_m=point.elevation,
                    time=point.time
                )
                if type(point.elevation) in [float, int]:
                    row['elevation_f'] = point.elevation * 3.28084
                else:
                    row['elevation_f'] = None
                rows.append(row)
    df = pd.DataFrame(rows)
    assert not np.all(df['elevation_m'].isna())
    return df

In [85]:
def bearing_deg(lat1, lon1, lat2, lon2):
    """
    Bearing from point 1 to point 2, degrees in [0, 360).
    Inputs in degrees.
    """
    lat1 = np.deg2rad(lat1)
    lon1 = np.deg2rad(lon1)
    lat2 = np.deg2rad(lat2)
    lon2 = np.deg2rad(lon2)
    dlon = lon2 - lon1

    y = np.sin(dlon) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(dlon)
    brng = np.rad2deg(np.arctan2(y, x))
    return (brng + 360) % 360

def step_data(df):
    rad_coords = df[['lat', 'lon']].apply(np.deg2rad)
    step_meters = haversine_distances(rad_coords.iloc[:-1], rad_coords.iloc[1:]).diagonal() * 6371000
    lat = df["lat"].to_numpy()
    lon = df["lon"].to_numpy()
    brng = bearing_deg(lat[:-1], lon[:-1], lat[1:], lon[1:])
    df['step_bearing'] = np.array([*brng, np.nan])
    df['step_bearing'] = df['step_bearing'].replace({0:np.nan})
    df['step_turn'] = df['step_bearing'].diff().abs()
    df['step_dist_m'] = np.array([0, *step_meters])
    df['step_dist_f'] = df['step_dist_m'] * 3.28084
    df['step_elevation_m'] = df['elevation_m'].diff().fillna(0)
    df['step_elevation_f'] = df['elevation_f'].diff().fillna(0)
    df['step_grade'] = df['step_elevation_m'] / df['step_dist_m'].replace(0, np.nan)
    return df

In [86]:
def step_analysis(df, rolling_window=3):
    df['avg_step_grade'] = df['step_grade'].rolling(rolling_window).mean()
    df['avg_bearing_change'] = df['step_turn'].rolling(rolling_window).mean() 
    hazards = dict(
        light_descent=(df["step_grade"].round(2) < -0.03) | (df["avg_step_grade"].round(2) < -0.08),
        steep_descent=(df["step_grade"].round(2) < -0.1) | (df["avg_step_grade"].round(2) < -0.15),
        ultra_steep_descent=(df["step_grade"].round(2) < -0.2) | (df["avg_step_grade"].round(2) < -0.2),
        turn_on_descent=(
            # Sudden Turn coming off a steep descent
            (df["step_turn"].round() > 24) & (df["step_grade"].shift(-1).round(2) < -0.05)
        ) |
        (
            # Sudden Turn on a steep descent
            (df["step_turn"].round() > 24) & (df["step_grade"] < -0.05)
        ) |
        (
            # Sustained Turn
            (df["avg_bearing_change"].round() > 24) & (df["avg_step_grade"].round(2) < -0.05)
            ),
        turn_on_steep_descent=(
            # Sudden Turn coming off a steep descent
            (df["step_turn"].round() > 24) & (df["step_grade"].shift(-1).round(2) < -0.1)
        ) |
        (
            # Sudden Turn on a steep descent
            (df["step_turn"].round() > 24) & (df["step_grade"].round(2) < -0.1)
        ) |
        (
            # Sustained Turn
            (df["avg_bearing_change"].round() > 24) & (df["avg_step_grade"].round(2) < -0.1)
            )
    )
    df["hazard"] = "none"
    for h in list(hazards.keys()):
        df.loc[hazards[h], "hazard"] = h
    return df

In [87]:
hazard_colors = {
    "light_descent": "#fee08b",
    "steep_descent": "#f46d43",
    "ultra_steep_descent": "#9F0712",
    "turn_on_descent": "#4F39F6",
    "turn_on_steep_descent": "#8A0194",
    "none": "#31D492"
}

In [88]:
def points_to_segments_lonlat(gdf, lon="lon", lat="lat", sort_col=None):
    g = gdf.sort_values(sort_col) if sort_col else gdf.sort_index()
    x0, y0 = g[lon].to_numpy()[:-1], g[lat].to_numpy()[:-1]
    x1, y1 = g[lon].to_numpy()[1:],  g[lat].to_numpy()[1:]
    seg_geom = [LineString([(a,b),(c,d)]) for a,b,c,d in zip(x0,y0,x1,y1)]
    seg = gpd.GeoDataFrame(
        {"start_i": g.index[:-1], "end_i": g.index[1:]},
        geometry=seg_geom,
        crs=gdf.crs
    )
    seg = seg.merge(gdf.drop(columns='geometry'), left_on='end_i', right_on='step', how='left')
    return seg

def google_maps_link(df, lat_col='lat', lon_col='lon', link_col='gmaps_url'):
    df[link_col] = (
        "https://www.google.com/maps?q="
        + df[lat_col].astype(str)
        + ","
        + df[lon_col].astype(str)
    )
    return df


In [89]:
paths = ["gpx_data\\onthegomap-16.4-mi-route (1).gpx"]

In [90]:
dfs = []
segments = []
for path in paths:
    df = read_simple_gpx(path)
    df['name'] = path.split(".gpx")[0]
    df = step_data(df)
    df = step_analysis(df)
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326")
    gdf_segments = points_to_segments_lonlat(gdf)
    segments.append(gdf_segments)
    dfs.append(df)
df = pd.concat(dfs, ignore_index=True)
gdf_segments = gpd.GeoDataFrame(
    pd.concat(segments, ignore_index=True),
    crs=segments[0].crs,
)

In [91]:
df[df['step_grade'] > 0].groupby('name')['step_grade'].describe()

,count,mean,std,min,25%,50%,75%,max
name,,,,,,,,
gpx_data\onthegomap-16.4-mi-route (1),366.0,0.054058,0.039614,0.001519,0.021041,0.042388,0.084762,0.156262


In [92]:
df[df['step_grade'] < 0].groupby('name')['step_grade'].describe()

,count,mean,std,min,25%,50%,75%,max
name,,,,,,,,
gpx_data\onthegomap-16.4-mi-route (1),544.0,-0.045368,0.029013,-0.167308,-0.06513,-0.040965,-0.022058,-0.001097


In [97]:
df

,name,step,lat,lon,elevation_m,time,elevation_f,step_bearing,step_turn,step_dist_m,step_dist_f,step_elevation_m,step_elevation_f,step_grade,avg_step_grade,avg_bearing_change,hazard
0,gpx_data\onthegomap-16.4-mi-route (1),0,37.83118,-122.25431,32.4,None,106.299216,284.209866,NaN,0.000000,0.000000,0.0,0.000000,NaN,NaN,NaN,none
1,gpx_data\onthegomap-16.4-mi-route (1),1,37.83119,-122.25436,32.5,None,106.627300,24.290892,259.918973,4.529804,14.861561,0.1,0.328084,0.022076,NaN,NaN,none
2,gpx_data\onthegomap-16.4-mi-route (1),2,37.83126,-122.25432,32.8,None,107.611552,29.429817,5.138925,8.539674,28.017305,0.3,0.984252,0.035130,NaN,NaN,none
3,gpx_data\onthegomap-16.4-mi-route (1),3,37.83133,-122.25427,33.0,None,108.267720,25.806531,3.623286,8.936876,29.320462,0.2,0.656168,0.022379,0.026528,89.560395,none
4,gpx_data\onthegomap-16.4-mi-route (1),4,37.83182,-122.25397,34.3,None,112.532812,26.775554,0.969023,60.521408,198.561055,1.3,4.265092,0.021480,0.026330,3.243745,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957,gpx_data\onthegomap-16.4-mi-route (1),957,37.83119,-122.25609,31.3,None,102.690292,89.999979,3.117301,81.797465,268.364396,1.0,3.280840,0.012225,0.004075,9.473219,none
958,gpx_data\onthegomap-16.4-mi-route (1),958,37.83119,-122.25602,31.4,None,103.018376,87.734124,2.265855,6.147688,20.169581,0.1,0.328084,0.016266,0.009497,5.491897,none
959,gpx_data\onthegomap-16.4-mi-route (1),959,37.83120,-122.25570,31.6,None,103.674544,86.849078,0.885046,28.125704,92.275935,0.2,0.656168,0.007111,0.011868,2.089401,none
960,gpx_data\onthegomap-16.4-mi-route (1),960,37.83121,-122.25547,31.8,None,104.330712,86.187422,0.661656,20.230125,66.371803,0.2,0.656168,0.009886,0.011088,1.270852,none


In [93]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326")

In [94]:
gdf_segments = points_to_segments_lonlat(gdf)

In [95]:
gdf_segments = google_maps_link(gdf_segments, lat_col='lat', lon_col='lon', link_col='gmaps_url')
gdf_segments["gmaps_link"] = (
    '<a href="https://www.google.com/maps?q='
    + gdf_segments["lat"].astype(str)
    + ','
    + gdf_segments["lon"].astype(str)
    + '" target="_blank">Open in Google Maps</a>'
)
gdf_segments["gmaps_popup"] = (
    '<b>Hazard:</b> ' + gdf_segments["hazard"].astype(str) + '<br>'
    '<a href="https://www.google.com/maps?q='
    + gdf_segments["lat"].astype(str)
    + ','
    + gdf_segments["lon"].astype(str)
    + '" target="_blank">📍 Open in Google Maps</a>'
)



In [96]:
m = gdf_segments.explore(
    column='hazard',
    tooltip=[
        'hazard'
    ],
    popup=[
        'hazard',
        'step_turn',
        'step_grade',
        'avg_step_grade',
        'avg_bearing_change',
        'gmaps_popup'
    ],
    categorical=True,
    cmap=list(hazard_colors.values()),
    categories=list(hazard_colors.keys()),
    legend=True,
    style_kwds={"weight": 6},
    escape=False
)
gdf.iloc[[0]].explore(m=m,  marker_type="marker",
    tooltip=False,
    marker_kwds={
        "icon": folium.Icon(
            icon="arrow-right",
            prefix="fa",   # Font Awesome
            color="green"
        )
    }
)
gdf.iloc[[-1]].explore(m=m,  marker_type="marker",
    tooltip=False,
    marker_kwds={
        "icon": folium.Icon(
            icon="stop",
            prefix="fa",   # Font Awesome
            color="red"
        )
    }
)
m


In [53]:
route_line = gdf_segments.union_all()
route_poly = (
    gpd.GeoSeries([route_line], crs=gdf_segments.crs)
    .to_crs(3857)
    .buffer(2)
    .to_crs(gdf_segments.crs)
    .iloc[0]
)

In [54]:
tags = {"highway": ["stop", "traffic_signals"]}
all_stops = ox.features_from_polygon(route_poly, tags=tags)

In [55]:
lights = all_stops[all_stops["highway"] == "traffic_signals"]
stops  = all_stops[all_stops["highway"] == "stop"]

In [56]:
stops

,,geometry,highway,traffic_signals:direction
element,id,,,


In [57]:
if 'description' in lights.columns:
    lights = lights[lights['description'] != 'flashing yellow light']
lights

geometry          highway  \
element id                                                       
node    53056317  POINT (-122.28823 37.88865)  traffic_signals   
        53056319  POINT (-122.28512 37.88951)  traffic_signals   
        53056333   POINT (-122.2789 37.88956)  traffic_signals   
        53056335  POINT (-122.27654 37.88888)  traffic_signals   

                 traffic_signals:direction  
element id                                  
node    53056317                      both  
        53056319                       NaN  
        53056333                       NaN  
        53056335                       NaN

In [ ]:
m = gdf_segments.explore(column='hazard', categorical=True, cmap=list(hazard_colors.values()), categories=list(hazard_colors.keys()), legend=True,style_kwds={"weight": 6})
gdf.iloc[[0]].explore(m=m,  marker_type="marker",
    tooltip=False,
    marker_kwds={
        "icon": folium.Icon(
            icon="arrow-right",
            prefix="fa",   # Font Awesome
            color="green"
        )
    }
)
gdf.iloc[[-1]].explore(m=m,  marker_type="marker",
    tooltip=False,
    marker_kwds={
        "icon": folium.Icon(
            icon="stop",
            prefix="fa",   # Font Awesome
            color="red"
        )
    }
)
stops.explore(
    m=m,
    tooltip=["highway"],
    marker_kwds={"radius": 4, "color": "red"},
)
lights.explore(
    m=m,
    tooltip=["highway"],
    marker_kwds={"radius": 6},
)
m


In [28]:
# import osmnx as ox
# import geopandas as gpd
# import numpy as np

# def controls_on_route_edges(
#     gdf_segments,
#     corridor_m=60,
#     ctrl_snap_max_m=10,
#     network_type="bike",
#     include_crossings=False,
# ):
#     """
#     Returns controls that apply to the route by:
#       - building a corridor polygon around your route
#       - downloading an OSMnx graph in that corridor
#       - snapping route segments to graph edges to get the route edge set
#       - snapping control points to graph edges
#       - keeping only controls whose snapped edge is in the route edge set

#     gdf_segments: GeoDataFrame of LineString route segments in EPSG:4326
#     """
#     # 1) Corridor polygon around the route (buffer in meters)
#     route_line = gdf_segments.union_all()
#     route_poly = (
#         gpd.GeoSeries([route_line], crs=gdf_segments.crs)
#         .to_crs(3857)
#         .buffer(corridor_m)
#         .to_crs(gdf_segments.crs)
#         .iloc[0]
#     )

#     # 2) Download and project graph
#     G = ox.graph_from_polygon(route_poly, network_type=network_type)
#     Gp = ox.project_graph(G, to_crs="EPSG:3857")

#     # 3) Snap route segments to nearest edges (use segment midpoints)
#     segs_3857 = gdf_segments.to_crs(3857).copy()
#     mids = segs_3857.geometry.interpolate(0.5, normalized=True)
#     uvk_route = ox.distance.nearest_edges(Gp, mids.x.to_numpy(), mids.y.to_numpy(), return_dist=False)
#     route_edge_set = set(uvk_route)

#     # 4) Fetch candidate controls (stop signs + traffic lights)
#     tags = {"highway": ["stop", "traffic_signals"]}
#     if include_crossings:
#         # optional: sometimes signals show up as crossing-related nodes
#         tags["highway"].append("crossing")

#     controls = ox.features_from_polygon(route_poly, tags=tags)
#     controls = controls[controls.geometry.type == "Point"].copy()

#     # 5) Snap controls to nearest edges and filter to route edges
#     controls_3857 = controls.to_crs(3857)
#     uvk_ctrl, d_ctrl = ox.distance.nearest_edges(
#         Gp, controls_3857.geometry.x.to_numpy(), controls_3857.geometry.y.to_numpy(), return_dist=True
#     )

#     controls_3857["u"] = [t[0] for t in uvk_ctrl]
#     controls_3857["v"] = [t[1] for t in uvk_ctrl]
#     controls_3857["key"] = [t[2] for t in uvk_ctrl]
#     controls_3857["snap_dist_m"] = d_ctrl

#     on_route = controls_3857[
#         controls_3857.apply(lambda r: (r["u"], r["v"], r["key"]) in route_edge_set, axis=1)
#         & (controls_3857["snap_dist_m"] <= ctrl_snap_max_m)
#         & (controls_3857["highway"].isin(["stop", "traffic_signals"]))  # keep only these two
#     ].copy()

#     return on_route.to_crs(gdf_segments.crs), controls.to_crs(gdf_segments.crs), G


In [29]:
# controls_on_route, controls_all, G = controls_on_route_edges(
#     gdf_segments,
#     corridor_m=60,
#     ctrl_snap_max_m=10,
#     network_type="bike",
# )

# # separate them
# stops  = controls_on_route[controls_on_route["highway"] == "stop"]
# lights = controls_on_route[controls_on_route["highway"] == "traffic_signals"]


In [30]:
# m = gdf_segments.explore(column='hazard', cmap='Reds', style_kwds={"weight": 6})
# controls_on_route.explore(
#     m=m,
#     tooltip=["highway", "snap_dist_m"],
#     marker_kwds={"radius": 6},
# )
# m

